In [2]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import sys 
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import plotly.graph_objs as go
#import plotly.plotly as py
import datetime as dt
import matplotlib.dates as mdates

In [3]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam, SGD

ModuleNotFoundError: No module named 'tensorflow.python'

In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
# import io
# df = pd.read_csv(io.StringIO(uploaded['crypto-markets.csv'].decode('utf-8')))

df = pd.read_csv("crypto-markets.csv")

data_copy = df

In [ ]:
df.head()

In [ ]:
df.shape #total number of rows and columns

In [ ]:
df.isnull().sum() #total of null values column wise 

In [ ]:
df.describe() #statistical readings of all the columns 

In [ ]:
df.info() #info on all the columns

In [ ]:
data = df #copy of dataframe

#set up a column with an index so I can only sample 1/10th of the dataset 
data["row_id"]= range(1, len(data) + 1)

In [ ]:
#drop columns I'm not using
data_=data.drop(['slug','ranknow','volume','market'], axis=1)
data_.head()

In [ ]:
data_.set_index('row_id', inplace=True)
data_.head()

In [ ]:
#counitng unique symbols 
data["name"].unique() #total number of unique cryptos 

In [ ]:
#set date to timestamp format
data_['date'] = pd.to_datetime(data_['date']).dt.date 

In [ ]:
#pick out the currency for two years span (2016>)

date = data_[data_['date'] >= dt.date(2016, 1, 1)]

In [ ]:
#show if each one closed up or down each day
date['pos_neg']= date['open']-date['close']
date.head()
#create a binary column - 0 = gain, 1 = loss to have something to predict
date['Up/Down'] = np.where(date['pos_neg']>0, '0', '1')

In [ ]:
#create data sets for different crypto currencies
etherdate = date[date['symbol']=='ETH']
ltcdate = date[date['symbol']=='LTC']
tetherdate = date[date['symbol']=='USDT']
cardanodate = date[date['symbol']=='ADA']
rippledate = date[date['symbol']=='XRP']

In [ ]:
#concat the different frames into one
frames=[etherdate,ltcdate,tetherdate,cardanodate,rippledate]
five_crypto = pd.concat(frames)

In [ ]:
five_crypto.info()

In [ ]:
#this chart shows each of the six and the number of times they closed up for the day.  
sns.set_style('whitegrid')
sns.countplot(x='symbol',hue='Up/Down',data=five_crypto,palette='rainbow')

In [ ]:
#a comparison the the up/down ratio at the end of the day for the six.
sns.set(style="whitegrid")

g = sns.factorplot("symbol", "close_ratio", "Up/Down",
                    data=five_crypto, kind="bar",
                    size=6, palette="muted",
                   legend_out=True)
g.despine(left=True)
g.set_ylabels("Ratio Count")

In [ ]:
data['market_billion'] = data['market'] / 1000000000
data['volume_million'] = data['volume'] / 1000000000
data['volume_billion'] = data['volume']

In [ ]:
# Let's prepare one dataframe where we will observe closing prices for each currency
wide_format = data.groupby(['date', 'name'])['close'].last().unstack()
wide_format.head(3)

In [ ]:
ax = data.groupby(['name'])['market_billion'].last().sort_values(ascending=False).head(10).sort_values().plot(kind='barh');
ax.set_xlabel("Market cap (in billion USD)");
plt.title("Top 10 Currencies by Market Cap");

In [ ]:
ax = data.groupby(['name'])['volume_million'].last().sort_values(ascending=False).head(10).sort_values().plot(kind='barh');
ax.set_xlabel("Transaction Volume (in million)");
plt.title("Top 10 Currencies by Transaction Volume");

In [ ]:
# For sake of convenience, let's define the top 6 currencies

top_6_currency_names = data.groupby(['name'])['market'].last().sort_values(ascending=False).head(6).index
data_top_6_currencies = data[data['name'].isin(top_6_currency_names)]
data_top_6_currencies.head(5)

In [ ]:
ax = data_top_6_currencies.groupby(['date', 'name'])['close'].mean().unstack().plot();
ax.set_ylabel("Price per 1 unit (in USD)");
plt.title("Price per unit currency");

In [ ]:
ax = data_top_6_currencies.groupby(['date', 'name'])['market_billion'].mean().unstack().plot();
ax.set_ylabel("Market Cap (in billion USD)");
plt.title("Market cap per unit currency");

In [ ]:
#co-relation between different cryptos
plt.figure(figsize=(14,8))
sns.heatmap(wide_format[top_6_currency_names].corr(),vmin=0, vmax=1, cmap='coolwarm', annot=True);

In [ ]:
data_top_6_currencies.head(5)

In [ ]:
data_top_6_currencies.drop(['ranknow'],axis=1)

In [ ]:
data_top_6_currencies.shape

#Bitcoin

In [ ]:
#bitcoin
df1 = data_top_6_currencies[data_top_6_currencies['name']=='Bitcoin']

df1.shape

In [ ]:
print(df1)

In [ ]:
df1.head()

In [ ]:
d = df1.iloc[:, 5:10]
print(d.head())
print(d.shape)

In [ ]:
plt.figure(figsize=(12,6))
sns.lineplot(x=d.index, y="open", data=d).set_title("Price of Bitcoin")

In [ ]:
data = d.iloc[:, 0]
print(len(data))
print(data)
hist = []
target = []
length = 90
for i in range(len(data)-length):
    x = data[i:i+length]
    y = data[i+length]
    hist.append(x)
    target.append(y)

In [ ]:
print(hist[1][89])
print(data[90])
print(target[0])

In [ ]:
hist = np.array(hist)
target = np.array(target)
target = target.reshape(-1,1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
hist_scaled = sc.fit_transform(hist)
target_scaled = sc.fit_transform(target)

In [ ]:
hist_scaled = hist_scaled.reshape((len(hist_scaled), length, 1))
print(hist_scaled.shape)

In [ ]:
X_train = hist_scaled[:1900,:,:]
X_test = hist_scaled[1900:,:,:]
y_train = target_scaled[:1900,:]
y_test = target_scaled[1900:,:]

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
model = tf.keras.Sequential()
model.add(layers.LSTM(units=32, return_sequences=True,
                  input_shape=(90,1), dropout=0.2))
model.add(layers.LSTM(units=32, return_sequences=True,
                  dropout=0.2))
model.add(layers.LSTM(units=32, dropout=0.2))
model.add(layers.Dense(units=1))
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
history = model.fit(X_train, y_train, epochs=30, batch_size=32)

In [ ]:
loss = history.history['loss']
epoch_count = range(1, len(loss) + 1)
plt.figure(figsize=(12,8))
plt.plot(epoch_count, loss, 'r--')
plt.legend(['Training Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show();

In [ ]:
pred = model.predict(X_test)
plt.figure(figsize=(12,8))
plt.plot(y_test, color='blue', label='Real')
plt.plot(pred, color='red', label='Prediction')
plt.title('Bitcoin Price Prediction')
plt.legend()
plt.show()

#XRP (Ripple)

In [ ]:
#bitcoin
df2 = data_top_6_currencies[data_top_6_currencies['name']=='XRP']

df2.shape

In [ ]:
print(df2)

In [ ]:
df2.head()

In [ ]:
d2 = df2.iloc[:, 5:10]
print(d2.head())
print(d2.shape)

In [ ]:
plt.figure(figsize=(12,6))
sns.lineplot(x=d2.index, y="open", data=d2).set_title("Price of Ripple")

In [ ]:
data = d2.iloc[:, 0]

print(data)

In [ ]:
hist = []
target = []
length = 90
for i in range(2042,len(data)-length):
    x = data[2042:i+length]
    y = data[i+length]
    hist.append(x)
    target.append(y)

In [ ]:
hist = np.array(hist)
target = np.array(target)
target = target.reshape(-1,1)

In [ ]:
hist = hist.reshape(-1,1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
hist_scaled = sc.fit_transform(hist)
target_scaled = sc.fit_transform(target)

In [ ]:
hist_scaled = hist_scaled.reshape((len(hist_scaled), length, 1))
print(hist_scaled.shape)

In [ ]:
X_train = hist_scaled[:1900,:,:]
X_test = hist_scaled[1900:,:,:]
y_train = target_scaled[:1900,:]
y_test = target_scaled[1900:,:]

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
model = tf.keras.Sequential()
model.add(layers.LSTM(units=32, return_sequences=True,
                  input_shape=(90,1), dropout=0.2))
model.add(layers.LSTM(units=32, return_sequences=True,
                  dropout=0.2))
model.add(layers.LSTM(units=32, dropout=0.2))
model.add(layers.Dense(units=1))
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
history = model.fit(X_train, y_train, epochs=30, batch_size=32)

In [ ]:
pred = model.predict(X_test)
plt.figure(figsize=(12,8))
plt.plot(y_test, color='blue', label='Real')
plt.plot(pred, color='red', label='Prediction')
plt.title('Ripple Price Prediction')
plt.legend()
plt.show()

#Ethereum

In [ ]:
#bitcoin
df3 = data_top_6_currencies[data_top_6_currencies['slug']=='ethereum']

df3.shape

In [ ]:
print(df3)

In [ ]:
df3.head()

In [ ]:
d3 = df3.iloc[:, 5:10]
print(d3.head())
print(d3.shape)

In [ ]:
plt.figure(figsize=(12,6))
sns.lineplot(x=d3.index, y="open", data=d3).set_title("Price of Ethereum")

In [ ]:
data = d3.iloc[:, 0]

print(data)

In [ ]:
hist = []
target = []
length = 90
for i in range(3986,len(data)-length):
    x = data[3986:i+length]
    y = data[i+length]
    hist.append(x)
    target.append(y)

In [ ]:
hist = np.array(hist)
target = np.array(target)
target = target.reshape(-1,1)

In [ ]:
hist = hist.reshape(-1,1)

In [ ]:
hist_scaled = hist_scaled.reshape((len(hist_scaled), length, 1))
print(hist_scaled.shape)

In [ ]:
X_train = hist_scaled[:1900,:,:]
X_test = hist_scaled[1900:,:,:]
y_train = target_scaled[:1900,:]
y_test = target_scaled[1900:,:]

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
model = tf.keras.Sequential()
model.add(layers.LSTM(units=32, return_sequences=True,
                  input_shape=(90,1), dropout=0.2))
model.add(layers.LSTM(units=32, return_sequences=True,
                  dropout=0.2))
model.add(layers.LSTM(units=32, dropout=0.2))
model.add(layers.Dense(units=1))
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
history = model.fit(X_train, y_train, epochs=30, batch_size=32)

In [ ]:
loss = history.history['loss']
epoch_count = range(1, len(loss) + 1)
plt.figure(figsize=(12,8))
plt.plot(epoch_count, loss, 'r--')
plt.legend(['Training Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show();

In [ ]:
pred = model.predict(X_test)
plt.figure(figsize=(12,8))
plt.plot(y_test, color='blue', label='Real')
plt.plot(pred, color='red', label='Prediction')
plt.title('Ethereum Price Prediction')
plt.legend()
plt.show()

#Bitcoin Cash (BCH)

In [ ]:

df4 = data_top_6_currencies[data_top_6_currencies['symbol']=='BCH']

df4.shape

In [ ]:
print(df4)

In [ ]:
df4.head()

In [ ]:
d4 = df4.iloc[:, 5:10]
print(d4.head())
print(d4.shape)

In [ ]:
plt.figure(figsize=(12,6))
sns.lineplot(x=d4.index, y="open", data=d4).set_title("Price of Bitcoin Cash")

In [ ]:
data = d4.iloc[:, 0]

print(data)

In [ ]:
hist = []
target = []
length = 90
for i in range(6775,len(data)-length):
    x = data[6775:i+length]
    y = data[i+length]
    hist.append(x)
    target.append(y)

In [ ]:
hist = np.array(hist)
target = np.array(target)
target = target.reshape(-1,1)

In [ ]:
hist = hist.reshape(1,-1)

In [ ]:
hist_scaled = hist_scaled.reshape((len(hist_scaled), length, 1))
print(hist_scaled.shape)

In [ ]:
X_train = hist_scaled[:1900,:,:]
X_test = hist_scaled[1900:,:,:]
y_train = target_scaled[:1900,:]
y_test = target_scaled[1900:,:]

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
model = tf.keras.Sequential()
model.add(layers.LSTM(units=32, return_sequences=True,
                  input_shape=(90,1), dropout=0.2))
model.add(layers.LSTM(units=32, return_sequences=True,
                  dropout=0.2))
model.add(layers.LSTM(units=32, dropout=0.2))
model.add(layers.Dense(units=1))
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
history = model.fit(X_train, y_train, epochs=30, batch_size=32)

In [ ]:
pred = model.predict(X_test)
plt.figure(figsize=(12,8))
plt.plot(y_test, color='blue', label='Real')
plt.plot(pred, color='red', label='Prediction')
plt.title('Bitcoin Cash Prediction')
plt.legend()
plt.show()

#EOS

In [ ]:
df5 = data_top_6_currencies[data_top_6_currencies['symbol']=='EOS']

df5.shape

In [ ]:
print(df5)

In [ ]:
df5.head()

In [ ]:
d5 = df5.iloc[:, 5:10]
print(d5.head())
print(d5.shape)

In [ ]:
plt.figure(figsize=(12,6))
sns.lineplot(x=d5.index, y="open", data=d5).set_title("Price of EOS")

In [ ]:
data = d5.iloc[:, 0]

print(data)

In [ ]:
hist = []
target = []
length = 90
for i in range(7270,len(data)-length):
    x = data[7270:i+length]
    y = data[i+length]
    hist.append(x)
    target.append(y)

In [ ]:
hist = np.array(hist)
target = np.array(target)
target = target.reshape(-1,1)

In [ ]:
hist = hist.reshape(1,-1)

In [ ]:
hist_scaled = hist_scaled.reshape((len(hist_scaled), length, 1))
print(hist_scaled.shape)

In [ ]:
X_train = hist_scaled[:1900,:,:]
X_test = hist_scaled[1900:,:,:]
y_train = target_scaled[:1900,:]
y_test = target_scaled[1900:,:]

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
model = tf.keras.Sequential()
model.add(layers.LSTM(units=32, return_sequences=True,
                  input_shape=(90,1), dropout=0.2))
model.add(layers.LSTM(units=32, return_sequences=True,
                  dropout=0.2))
model.add(layers.LSTM(units=32, dropout=0.2))
model.add(layers.Dense(units=1))
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
history = model.fit(X_train, y_train, epochs=30, batch_size=32)

In [ ]:
pred = model.predict(X_test)
plt.figure(figsize=(12,8))
plt.plot(y_test, color='blue', label='Real')
plt.plot(pred, color='red', label='Prediction')
plt.title('EOS Prediction')
plt.legend()
plt.show()

#Stellar

In [ ]:
df6 = data_top_6_currencies[data_top_6_currencies['name']=='Stellar']

df6.shape

In [ ]:
print(df6)

In [ ]:
df6.head()

In [ ]:
d = df6.iloc[:, 5:10]
print(d.head())
print(d.shape)

In [ ]:
plt.figure(figsize=(12,6))
sns.lineplot(x=d.index, y="open", data=d).set_title("Price of Stellar")

In [ ]:
print(d)

In [ ]:
data = d.iloc[:, 0]
print(len(data))
print(data)
hist = []
target = []
length = 90
for i in range(5197, len(data)-length):
    x = data[5197,:i+length]
    y = data[i+length]
    hist.append(x)
    target.append(y)

In [ ]:
hist = np.array(hist)
target = np.array(target)
target = target.reshape(-1,1)
hist=hist.reshape(-1,1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
hist_scaled = sc.fit_transform(hist)
target_scaled = sc.fit_transform(target)

In [ ]:
hist_scaled = hist_scaled.reshape((len(hist_scaled), length, 1))
print(hist_scaled.shape)

In [ ]:
X_train = hist_scaled[:1900,:,:]
X_test = hist_scaled[1900:,:,:]
y_train = target_scaled[:1900,:]
y_test = target_scaled[1900:,:]

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
model = tf.keras.Sequential()
model.add(layers.LSTM(units=32, return_sequences=True,
                  input_shape=(90,1), dropout=0.2))
model.add(layers.LSTM(units=32, return_sequences=True,
                  dropout=0.2))
model.add(layers.LSTM(units=32, dropout=0.2))
model.add(layers.Dense(units=1))
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
history = model.fit(X_train, y_train, epochs=30, batch_size=32)

In [ ]:
pred = model.predict(X_test)
plt.figure(figsize=(12,8))
plt.plot(y_test, color='blue', label='Real')
plt.plot(pred, color='red', label='Prediction')
plt.title('Stellar Price Prediction')
plt.legend()
plt.show()